In [1]:
#python scripts/scripted_collect.py -n 100 -t 30 -e Widow250PickPlace-v1 -pl grasp -a grasp_success_target --noise=0.1

In [2]:
#python scripts/scripted_collect.py -n 100 -t 30 -e Widow250PickPlace-v0 -pl pickplace -a place_success_target --noise=0.1 --gui

In [3]:
#python scripts/scripted_collect.py -n 100 -t 30 -e Widow250PickPlaceMultiObject-v0 -pl pickplace -a place_success_target --noise=0.1 --gui

In [1]:
import numpy as np
import roboverse
from roboverse.policies import policies


def collect_data(env, model, policy, target, num_trajectories=100, num_timesteps=30):
    policy_class = policies[policy]
    policy = policy_class(env)
    num_success = 0
    num_saved = 0
    num_attempts = 0
    accept_trajectory_key = target
    noise = 0.1
    EPSILON = 0.1

    while num_saved < num_trajectories:
        num_attempts += 1
        num_steps = -1
        rewards = []
        env.reset()
        policy.reset()
        for j in range(num_timesteps):
            action, agent_info = policy.get_action()

            # In case we need to pad actions by 1 for easier realNVP modelling 
            env_action_dim = env.action_space.shape[0]
            #if env_action_dim - action.shape[0] == 1:
            #    action = np.append(action, 0)
            action += np.random.normal(scale=noise, size=(env_action_dim,))
            action = np.clip(action, -1 + EPSILON, 1 - EPSILON)
            observation = env.get_observation_stacked() #env.get_observation()
            next_observation, reward, done, info = env.step(action)
            #if not info[accept_trajectory_key]:
            #    reward += 0.99**(num_timesteps-j)/10
            rewards.append(reward)
            model.replay_buffer.add(observation, next_observation, action, reward, done, [{}])

            if info[accept_trajectory_key] and num_steps < 0:
                num_steps = j

            if info[accept_trajectory_key] and j > 18:
                break
            if done or agent_info['done']:
                break

        if info[accept_trajectory_key]:
            if True:
                print("num_timesteps: ", num_steps)
                #print(traj["observations"])
            num_success += 1
            num_saved += 1
        print(f"num_trajectories: {num_saved} success rate: {num_success/num_attempts} Reward: {sum(rewards)}")

    print("success rate: {}".format(num_success / (num_attempts)))

pybullet build time: Oct 14 2023 15:44:17


In [2]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise


env = roboverse.make("Widow250PickPlace-v1",
                         gui=False,
                         transpose_image=False)
obs = env.reset()

# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

# optimize_memory_usage=True,
model = TD3("MultiInputPolicy", env, buffer_size=50000, action_noise=action_noise, learning_rate=0.001, \
            tensorboard_log="data/td3", verbose=1, learning_starts=0) #noise Required for deterministic policy

collect_data(env, model, "grasp", "grasp_success_target", 1000, 30)
model.save_replay_buffer(f"data/td3_expert_grasp")

print("start pre-training from buffer only")
model.learn(total_timesteps=0, log_interval=5, tb_log_name="exp", progress_bar=True)
model.train(gradient_steps=10000)

print("start learning")
for i in range(5):
    model.learn(total_timesteps=20000, log_interval=10, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
    model.save("data/td3_1")
    #model.replay_buffer.reset()
    #model.load_replay_buffer(f"data/td3_expert_grasp")

print("finish learning")

/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (4, 48, 48, 3)
  logger.warn(
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passiv

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array

num_timesteps:  15
num_trajectories: 1 success rate: 1.0 Reward: 5.0
num_timesteps:  15
num_trajectories: 2 success rate: 1.0 Reward: 5.0
num_timesteps:  14
num_trajectories: 3 success rate: 1.0 Reward: 6.0
num_timesteps:  12
num_trajectories: 4 success rate: 1.0 Reward: 8.0
num_timesteps:  17
num_trajectories: 5 success rate: 1.0 Reward: 3.0
num_trajectories: 5 success rate: 0.8333333333333334 Reward: 0.0
num_timesteps:  15
num_trajectories: 6 success rate: 0.8571428571428571 Reward: 5.0
num_timesteps:  15
num_trajectories: 7 success rate: 0.875 Reward: 5.0
num_timesteps:  14
num_trajectories: 8 success rate: 0.8888888888888888 Reward: 6.0
num_timesteps:  16
num_trajectories: 9 success rate: 0.9 Reward: 4.0
num_timesteps:  13
num_trajectories: 10 success rate: 0.9090909090909091 Reward: 7.0
num_timesteps:  12
num_trajectories: 11 success rate: 0.9166666666666666 Reward: 8.0
num_timesteps:  13
num_trajectories: 12 success rate: 0.9230769230769231 Reward: 7.0
num_timesteps:  24
num_traj

Output()

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 199      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 210      |
|    fps             | 22       |
|    time_elapsed    | 69       |
|    total_timesteps | 41790    |
| train/             |          |
|    actor_loss      | 11.7     |
|    critic_loss     | 1.08     |
|    learning_rate   | 0.001    |
|    n_updates       | 51591    |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 199      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 220      |
|    fps             | 22       |
|    time_elapsed    | 160      |
|    total_timesteps | 43780    |
| train/             |          |
|    actor_loss      | 10.8     |
|    critic_loss     | 0.959    |
|    learning_rate   | 0.001    |
|    n_updates       | 53581    |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 199      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 230      |
|    fps             | 22       |
|    time_elapsed    | 252      |
|    total_timesteps | 45770    |
| train/             |          |
|    actor_loss      | 9.92     |
|    critic_loss     | 0.896    |
|    learning_rate   | 0.001    |
|    n_updates       | 55571    |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 199      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 240      |
|    fps             | 21       |
|    time_elapsed    | 344      |
|    total_timesteps | 47760    |
| train/             |          |
|    actor_loss      | 9.16     |
|    critic_loss     | 0.762    |
|    learning_rate   | 0.001    |
|    n_updates       | 57561    |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 199      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 250      |
|    fps             | 21       |
|    time_elapsed    | 435      |
|    total_timesteps | 49750    |
| train/             |          |
|    actor_loss      | 8.5      |
|    critic_loss     | 0.632    |
|    learning_rate   | 0.001    |
|    n_updates       | 59551    |
---------------------------------


In [2]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise


COLLECT = True
#env = gym.make("Pendulum-v1", render_mode="rgb_array")
env = roboverse.make("Widow250PickPlace-v1",
                         gui=True,
                         transpose_image=False)
obs = env.reset()

# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

# optimize_memory_usage=True,
model = TD3("MultiInputPolicy", env, buffer_size=25000, action_noise=action_noise, learning_rate=0.0001, \
            tensorboard_log="data/td3", verbose=1, learning_starts=0) #noise Required for deterministic policy
if COLLECT:
    for i in range(2):
        collect_data(env, model, "grasp", "grasp_success_target", 250, 30)
        model.save_replay_buffer(f"data/td3_expert_grasp{i+1}")

if not COLLECT:
    print("start pre-training from buffer only")
    model.replay_buffer.reset()
    for i in range(2):
        model.load_replay_buffer(f"data/td3_expert_grasp{i%2+1}")
    model.learn(total_timesteps=0, log_interval=5, tb_log_name="exp", progress_bar=True)
    model.train(gradient_steps=2000)

    print("start learning")
    for i in range(20):
        model.learn(total_timesteps=2005, log_interval=5, tb_log_name="exp", progress_bar=True)
        if (i+1)%10 == 0:
            model.replay_buffer.reset()
            model.load_replay_buffer(f"data/td3_expert_grasp1")
            model.load_replay_buffer(f"data/td3_expert_grasp2")
            model.save("data/td3_1")

    print("finish learning")

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (Intel(R) UHD Graphics 630)
GL_VERSION=4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.10
pthread_getconcurrency()=0
Version = 4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (Intel(R) UHD Graphics 630)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Microsoft Corporation
ven = Microsoft Corporation


/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (4, 48, 48, 3)
  logger.warn(
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passiv

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array

num_timesteps:  19
num_trajectories: 1 success rate: 1.0 Reward: 1.0
num_timesteps:  14
num_trajectories: 2 success rate: 1.0 Reward: 6.0
num_timesteps:  13
num_trajectories: 3 success rate: 1.0 Reward: 7.0
num_timesteps:  16
num_trajectories: 4 success rate: 1.0 Reward: 4.0
num_timesteps:  15
num_trajectories: 5 success rate: 1.0 Reward: 5.0
num_timesteps:  17
num_trajectories: 6 success rate: 1.0 Reward: 3.0
num_timesteps:  14
num_trajectories: 7 success rate: 1.0 Reward: 6.0
num_timesteps:  15
num_trajectories: 8 success rate: 1.0 Reward: 4.0
num_timesteps:  15
num_trajectories: 9 success rate: 1.0 Reward: 5.0
num_timesteps:  15
num_trajectories: 10 success rate: 1.0 Reward: 5.0
num_timesteps:  16
num_trajectories: 11 success rate: 1.0 Reward: 4.0
num_timesteps:  13
num_trajectories: 12 success rate: 1.0 Reward: 7.0
num_timesteps:  17
num_trajectories: 13 success rate: 1.0 Reward: 3.0
num_timesteps:  15
num_trajectories: 14 success rate: 1.0 Reward: 5.0
num_timesteps:  15
num_trajec

In [2]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise


model = TD3.load("data/td3_1")

In [6]:
# del model # remove to demonstrate saving and loading
# model = TD3.load("data/td3_1")

# start env with gui
env.close()
env = roboverse.make("Widow250PickPlace-v1",
                         gui=True,
                         transpose_image=False)
obs = env.reset()
model.set_env(env)
env = model.get_env()

obs = env.reset()
print("start render")
for i in range(int(1e4)):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render("human")
env.close()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (Intel(R) UHD Graphics 630)
GL_VERSION=4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.10
pthread_getconcurrency()=0
Version = 4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (Intel(R) UHD Graphics 630)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Microsoft Corporation
ven = Microsoft Corporation
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
start r

/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:225: UserWarning: You tried to render a VecEnv with mode='human' but the render mode defined when initializing the environment must be 'human' or 'rgb_array', not 'None'.
  warnings.warn(


KeyboardInterrupt: 